# Loading data to Lancedb

In [12]:
import lancedb
class LanceDB:
    def __init__(self):
        self.db = lancedb.connect("./.lancedb")

    def list_tables(self):
        return self.db.table_names()
    
    def load_data_from_table(self,table_name):
        db_table = self.db.open_table(table_name)
        return db_table.to_pandas()


### Load Environment variables

In [22]:
import os
from dotenv import load_dotenv
path = os.path.join('./','.env')
load_dotenv(path)

# Get the API key
notion_api_key = os.getenv('SOURCES__REST_API__NOTION__API_KEY')

### Load data from Notion "Homework: Employee handbook" page to LanceDB 'homework' table using dlt

In [24]:
import dlt
from rest_api import RESTAPIConfig, rest_api_source

from dlt.sources.helpers.rest_client.paginators import BasePaginator, JSONResponsePaginator
from dlt.sources.helpers.requests import Response, Request

from dlt.destinations.adapters import lancedb_adapter

In [25]:
from datetime import datetime, timezone

class PostBodyPaginator(BasePaginator):
    def __init__(self):
        super().__init__()
        self.cursor = None

    def update_state(self, response: Response) -> None:
        # Assuming the API returns an empty list when no more data is available
        if not response.json():
            self._has_next_page = False
        else:
            self.cursor = response.json().get("next_cursor")
            if self.cursor is None:
                self._has_next_page = False

    def update_request(self, request: Request) -> None:
        if request.json is None:
            request.json = {}

        # Add the cursor to the request body
        request.json["start_cursor"] = self.cursor

@dlt.resource(name="homework")
def rest_api_notion_resource():
    notion_config: RESTAPIConfig = {
        "client": {
            "base_url": "https://api.notion.com/v1/",
            "auth": {
                "token": notion_api_key
            },
            "headers":{
            "Content-Type": "application/json",
            "Notion-Version": "2022-06-28"
            }
        },
        "resources": [
            {
                "name": "search",
                "endpoint": {
                    "path": "search",
                    "method": "POST",
                    "paginator": PostBodyPaginator(),
                    "json": {
                        "query": "Homework: Employee handbook",
                        "sort": {
                            "direction": "ascending",
                            "timestamp": "last_edited_time"
                        }
                    },
                    "data_selector": "results"
                }
            },
            {
                "name": "page_content",
                "endpoint": {
                    "path": "blocks/{page_id}/children",
                    "paginator": JSONResponsePaginator(),
                    "params": {
                        "page_id": {
                            "type": "resolve",
                            "resource": "search",
                            "field": "id"
                        }
                    },
                }
            }
        ]
    }

    yield from rest_api_source(notion_config,name="homework")

def extract_page_content(response):
    block_id = response["id"]
    last_edited_time = response["last_edited_time"]
    block_type = response.get("type", "Not paragraph")
    if block_type != "paragraph":
        content = ""
    else:
        try:
            content = response["paragraph"]["rich_text"][0]["plain_text"]
        except IndexError:
            content = ""
    return {
        "block_id": block_id,
        "block_type": block_type,
        "content": content,
        "last_edited_time": last_edited_time,
        "inserted_at_time": datetime.now(timezone.utc)
    }

@dlt.resource(
    name="homework",
    write_disposition="merge",
    primary_key="block_id",
    columns={"last_edited_time":{"dedup_sort":"desc"}}
    )
def rest_api_notion_incremental(
    last_edited_time = dlt.sources.incremental("last_edited_time", initial_value="2024-06-26T08:16:00.000Z",primary_key=("block_id"))
):
    # last_value = last_edited_time.last_value
    # print(last_value)

    for block in rest_api_notion_resource.add_map(extract_page_content):
        if not(len(block["content"])):
            continue
        yield block

def load_notion() -> None:
    pipeline = dlt.pipeline(
        pipeline_name="company_policies",
        destination="lancedb",
        dataset_name="notion_pages",
        # full_refresh=True
    )

    load_info = pipeline.run(
        lancedb_adapter(
            rest_api_notion_incremental,
            embed="content"
        ),
        table_name="homework",
        write_disposition="merge"
    )
    print(load_info)

load_notion()

Pipeline company_policies load step completed in ---
0 load package(s) were loaded to destination LanceDB and into dataset None
The LanceDB destination used <dlt.destinations.impl.lancedb.configuration.LanceDBCredentials object at 0x2c4f78950> location to store data


In [26]:
ldb = LanceDB()
ldb.list_tables()

['notion_pages____dlt_loads',
 'notion_pages____dlt_pipeline_state',
 'notion_pages____dlt_version',
 'notion_pages___dltSentinelTable',
 'notion_pages___employee_handbook',
 'notion_pages___homework']

### Question 1

In [27]:
df = ldb.load_data_from_table('notion_pages___homework')
print(f"The number of rows in notion_pages__homework is {df.shape[0]}")
df

The number of rows in notion_pages__homework is 17


,id__,vector__,block_id,block_type,content,last_edited_time,inserted_at_time,_dlt_load_id,_dlt_id
0,c69f1ecf-7b02-5810-8286-3f42659ae9d4,"[-0.024265625, 0.04746082, -0.0117964735, 0.06...",a8196881-ae94-4767-8767-92fe1a327d24,paragraph,We owe our success to our employees. To show o...,2024-07-05 22:34:00+00:00,2024-07-22 06:48:28.383465+00:00,1721630901.815667,L+prfRIgPdvqeQ
1,f2c18ac0-50f5-5b72-a871-dc5a46780353,"[-0.049661595, 0.10853516, -0.0097626075, -0.0...",31fcbf26-2ca5-468a-8af8-d7eb4c2db8c8,paragraph,We want to ensure that private information abo...,2024-07-05 22:38:00+00:00,2024-07-22 06:48:28.387057+00:00,1721630901.815667,2fCsRZ/SLCWTkQ
2,4553193e-c655-54df-9a33-cfc570bf34d0,"[-0.06316321, 0.1733151, 0.02535164, -0.019146...",da7721fd-3d0f-4c04-bc5e-825ad60bed1c,paragraph,Employee health is important to us. We don’t d...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387197+00:00,1721630901.815667,MMubDwdpt0DTsg
3,791be1a1-6c67-530d-87ab-bd9912500ea5,"[-0.10974312, 0.105860755, 0.003290652, -0.021...",ff36dcf3-5faa-40b4-ad8e-92fdc952201e,paragraph,Our company is dedicated to maintaining a safe...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387318+00:00,1721630901.815667,HyTw20hxHgxQ1w
4,a83497f4-922c-5d62-bab1-53804e93c811,"[0.052423332, -0.06457597, 0.065863006, 0.0145...",a1ff9697-4bb6-4f1e-b464-dda296dbd307,paragraph,If your job doesn’t require you to be present ...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387445+00:00,1721630901.815667,8MkT3xl2hg+mug
5,434b71e9-a11a-519d-a9fe-e3ade78d47d6,"[0.0005233809, -0.05488343, 0.043573394, -0.01...",e4ec9f4d-b687-4c28-a80d-985bfabcc2ba,paragraph,Remote working refers to working from a non-of...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387567+00:00,1721630901.815667,/lvF+YWBoZcIjw
6,17816363-54b7-5ba7-b8d5-06d871a25414,"[0.03802628, -0.02150967, 0.04752783, 0.064706...",e6e550dc-b59e-4928-abd7-07eace948681,paragraph,There are some expenses that we will pay direc...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387684+00:00,1721630901.815667,2QK4g58FLD1r9Q
7,2a434cf9-09d9-5514-a88b-02977f2f953e,"[-0.058588043, -0.07540445, 0.03377517, 0.0096...",a269d0ca-ce14-481b-a5f4-9192d6840d6e,paragraph,Our company operates between 9 a.m. to 7 p.m. ...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.387806+00:00,1721630901.815667,z3gm2vbLTbzkRA
8,5f9384fa-7f98-5f52-a06e-05b05f42f69a,"[-0.013599243, 0.0047530453, 0.024835166, 0.01...",5b65f3e7-0a37-429a-818d-f99b53755ebd,paragraph,"In this section, we are going to be covering i...",2024-07-05 23:33:00+00:00,2024-07-22 06:48:28.387959+00:00,1721630901.815667,JoJxbxoYiF8Jwg
9,42af72f6-9db7-54a2-87b2-d466169078ff,"[0.03206087, 0.024244677, 0.008471335, 0.03179...",b27f7d80-f2f1-460e-aa0c-b8e770cf050a,paragraph,Our company observes the following holidays: N...,2024-07-05 22:52:00+00:00,2024-07-22 06:48:28.388071+00:00,1721630901.815667,vBb4mQolL9VEYg


### Question 2

In [16]:
df['last_edited_time'].max()

Timestamp('2024-07-05 23:33:00+0000', tz='UTC')

### Question 3

In [17]:
def retrieve_context_from_lancedb(dbtable, question, top_k=2):

    query_results = dbtable.search(query=question).to_list()
    context = "\n".join([result["content"] for result in query_results[:top_k]])

    return context

In [18]:
import ollama
def main():
  # Connect to the lancedb table
  db = lancedb.connect(".lancedb")
  dbtable = db.open_table("notion_pages___homework")

  # A system prompt telling ollama to accept input in the form of "Question: ... ; Context: ..."
  messages = [
      {"role": "system", "content": "You are a helpful assistant that helps users understand policies inside a company's employee handbook. The user will first ask you a question and then provide you relevant paragraphs from the handbook as context. Please answer the question based on the provided context. For any details missing in the paragraph, encourage the employee to contact the HR for that information. Please keep the responses conversational."}
  ]

  while True:
    # Accept user question
    question = input("You: ")

    # Retrieve the relevant paragraphs on the question
    context = retrieve_context_from_lancedb(dbtable,question,top_k=2)

    # Create a user prompt using the question and retrieved context
    messages.append(
        {"role": "user", "content": f"Question: '{question}'; Context:'{context}'"}
    )

    # Get the response from the LLM
    response = ollama.chat(
        model="llama2-uncensored",
        messages=messages
    )
    response_content = response['message']['content']
    print(f"Assistant: {response_content}")

    # Add the response into the context window
    messages.append(
        {"role": "assistant", "content":response_content}
    )

In [19]:
main()

Assistant: Hello there! I hope you had a great day today and you enjoyed the information about paid time off (PTO) in our company handbook. 
As per your query, an employee is entitled to 30 days of PTO (Paid Time Off) per year. This means that they will receive 2.5 days per month during their first year with the company and can use any unused time off after the end of their first week with us. Additionally, they earn one additional day per year after their first year with our company, with a cap at 25 days overall.
If an employee needs to take PTO, they should send a request through our HRIS (Human Resources Information System) and if their manager or HR approves, the leave will be granted. There is no need to specify a reason for requesting PTO as this is considered part of your employment benefits. 
If an employee leaves our company before using all their accrued PTO days, we may compensate them for unused time off with their final paycheck according to local law. If there are no pro

KeyboardInterrupt: 